In [1]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [2]:
#Import libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk

In [3]:
#Import files
from data import refCoin, trainCoin, testCoin
import constants
importlib.reload(constants)

%load_ext autoreload
%autoreload 2

# 1. Load different Datasets

In [4]:
ref_data = refCoin()
train_data = trainCoin()
test_data = testCoin()

Loading data from pickle files


In [5]:
ref_data.raw_data.shape

(2, 4000, 6000, 3)

In [20]:
def find_contour(images: np.ndarray):
    """
    Find the contours for the set of images
    
    Args
    ----
    images: np.ndarray (N, 4000, 6000, 3)
        Source images to process

    Return
    ------
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    """

    # Get number of images to process
    N, _, _, _ = np.shape(images)

    # Fill in dummy values (fake points)
    contours = [np.array([[0, 0], [1, 1]]) for _ in range(N)]
    for idx, img in enumerate(images):
        gray_img = sk.color.rgb2gray(img)
        cnts = sk.measure.find_contours(np.transpose(gray_img), fully_connected='high')
        contours[idx] = cnts
    
    return contours
    

In [21]:
contours = find_contour(ref_data.raw_data)